In [24]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
corpus = gutenberg.sents()
gutenbergsentlist=list(corpus)
from collections import defaultdict
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import tqdm

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aksha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
file = open("wikidata.txt", "r",encoding="utf8")
contents = file.read()
wikisentlist=contents.split(".")
#each word in given analogy pair contain a sentences nearly 20 it may exceed by 3-5 depending on word that appear in other word senteces

In [3]:
def clean_corpus(words):
   stop_words = set(stopwords.words('english'))
   pattern = re.compile("^[a-zA-Z]+$")
   readycorpus=[]
   for ch in words:
       ch = ch.lower()
       if pattern.match(ch) and ch not in stop_words and ch != '.' and ch !=" ":
            readycorpus.append(ch)
   return readycorpus

In [4]:
wiki_window_corpus=[]
fullcorpus=[]
for sent in wikisentlist[0::
    rawsent=sent.split(" ")
    cleaned_raw= clean_corpus(rawsent)
    if len(cleaned_raw)>=3:
        fullcorpus+=cleaned_raw
        wiki_window_corpus.append(clean_corpus(rawsent)) 

In [5]:
vocabulary = list(sorted(set(fullcorpus)))

In [6]:
freq_dist = nltk.FreqDist(word for word in fullcorpus)
print("Total words in corpus: ",len(fullcorpus) )
print("Size of vocabulary: ",len(freq_dist))
print("Most frequent tokens: ",freq_dist.most_common(20))
V=len(freq_dist)

Total words in corpus:  33794
Size of vocabulary:  6453
Most frequent tokens:  [('city', 236), ('india', 193), ('largest', 186), ('world', 172), ('capital', 171), ('also', 167), ('europe', 155), ('population', 145), ('first', 143), ('war', 142), ('state', 138), ('became', 132), ('century', 128), ('country', 125), ('australia', 113), ('asia', 106), ('spain', 105), ('government', 103), ('modern', 103), ('one', 102)]


In [7]:
def indices_info(processed_corpus):
    processed =processed_corpus
    index = 0
    wordxInd = {}
    Indxword = {}
    for word in processed:
        if word not in wordxInd: 
             wordxInd[word] = index
             Indxword[index] = word
             index += 1
    return wordxInd, Indxword

In [8]:
import itertools
wordxInd, Indxword = indices_info(vocabulary)
print(dict(itertools.islice(wordxInd.items(), 5)))
print(dict(itertools.islice(Indxword.items(), 5)))

{'abandoned': 0, 'abbreviated': 1, 'abbreviation': 2, 'abdication': 3, 'abjad': 4}
{0: 'abandoned', 1: 'abbreviated', 2: 'abbreviation', 3: 'abdication', 4: 'abjad'}


In [9]:
import pandas as pd
df = pd.DataFrame({'vocab': vocabulary})
one_hot_encoded = pd.get_dummies(df['vocab'])

In [10]:
def get_windows(corpus, C):
        for words in corpus:
           i = 0
           while i < len(words):
              center_word = words[i]
              iC=0
              if (i-C)>=0:
                  iC=i-C
              context_words = words[iC:i] + words[(i+1):(i+C+1)]
              yield context_words, center_word
              i += 1

In [31]:
leng=0
for context,center in get_windows(wiki_window_corpus, 2):
    leng+=1
    if leng<5:
        print(f"context->{context}  center -> {center} ")

context->['municipality', 'largest']  center -> athens 
context->['athens', 'largest', 'population']  center -> municipality 
context->['athens', 'municipality', 'population', 'size']  center -> largest 
context->['municipality', 'largest', 'size', 'greece']  center -> population 


In [25]:
def get_x(corpus, C):
    train_x,train_y=None,None
    for context,center in tqdm.notebook.tqdm(get_windows(corpus, 2)):
        center_vector=one_hot_encoded.iloc[:,wordxInd[center]].to_numpy().reshape(V,1)
        context_vector=None
        for word in context:    
            col_vec = one_hot_encoded.iloc[:,wordxInd[word]].to_numpy().reshape(V,1)
            if context_vector is None:
                 context_vector = col_vec
            else:
                 context_vector = np.hstack((context_vector, col_vec))
        context_vector=np.mean(context_vector, axis=1,keepdims=True)
        context_vector=context_vector.reshape(V,1)
        if train_x is None:
            train_x = context_vector
            train_y = center_vector
        else:
            train_x =  np.hstack((train_x, context_vector))
            train_y =  np.hstack((train_y, center_vector))
    return train_x,train_y

In [ ]:
x,y=get_x(wiki_window_corpus, 2)
x=np.around(x,decimals=3)